# Visualize artifacts and the detections on them 

### Setup main notebook options

In [1]:
region_name = "TDS_ELAISN1"  #_ELAISN1

# Add artifacts here
dd_arts = [{"field_id":"GNU2597312684324028416", "ra":243.31, "dec":55.79, "rad":0.01},   # 0 Detection structures next to bright source
           {"field_id":"GNU2597312684324028416", "ra":243.28, "dec":55.76, "rad":0.02},   # 1 Multiple circular artifacts due to bright star
           {"field_id":"GNU2597312684324028416", "ra":243.41, "dec":56.27, "rad":0.02},   # 2 Edge line
           {"field_id":"GNU2597523790556561408", "ra":241.405, "dec":55.615, "rad":0.01}, # 3 Ring without bright source
           {"field_id":"GNU2597523790556561408", "ra":241.58, "dec":55.43, "rad":0.01},   # 4 Line artifact
           {"field_id":"GNU2597594159300739072", "ra":244.30, "dec":53.695, "rad":0.02},  # 5 Edge line
           {"field_id":"GNU2597664528044916736", "ra":242.79, "dec":53.97, "rad":0.01},   # 6 Ghost next to bright source
           {"field_id":"GFU2597312684324028416", "ra":243.36, "dec":55.75, "rad":0.07},   # 7 Very bright star makes cross, also effects in NUV idx 0,1
           #{"field_id":"", "ra":, "dec":, "rad":0.01},
           #{"field_id":"", "ra":, "dec":, "rad":0.01},
           #{"field_id":"", "ra":, "dec":, "rad":0.01}, 
        ]

In [2]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

from regions import CircleSkyRegion
from astropy import units as uu
from astropy.coordinates import SkyCoord
from astropy.wcs.utils import pixel_to_skycoord
import numpy as np

from vasca.region import Region
from vasca.field import BaseField
import vasca.visualization as vvis

from vasca.utils import mag2flux, sel_sources, dd_filter2id

#Load region and field
region_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+".fits"
rg = Region()
rg.load_from_fits(region_fname)
#display(rg.tt_fields)

In [3]:
def plot_art(art_idx):
    dd_art = dd_arts[art_idx]
    fd = rg.get_field(field_id=dd_art["field_id"], load_method="FITS", add_field=True)
    fig_art = plt.figure("Sky map artifact",figsize=(8, 7))
    
    sky_center = SkyCoord(dd_art["ra"]*uu.deg, dd_art["dec"]*uu.deg, frame="icrs")
    sky_art= CircleSkyRegion(center=sky_center, radius=dd_art["rad"]*uu.deg)
    plt.cla()
    ax, wcs_art = vvis.plot_field_sky_map(fd, img_idx=-1, sky_region = sky_art, fig=fig_art)
    
    print("all:red, ellipse:blue , class_star:green, artifact:white")
    lw= 0.8
    sel_s2n = fd.tt_detections["s2n"]>3.5
    fd.tt_detections["sel"]  = sel_s2n
    kdet_s2n = {"marker":"o","color":"r","markersize": 8,"alpha":1,"markeredgewidth": lw}
    ax, tt_det = vvis.plot_sky_sources( fd.tt_detections, src_id=None,src_kwargs=kdet_s2n, sky_region_wcs = (sky_art,wcs_art), only_selected=True)
    
    sel_ell = tt_det["ellip_world"]>0.5
    tt_det["sel"]  = sel_ell
    kdet_ell = {"marker":"o","color":"b","markersize": 6.5,"alpha":1,"markeredgewidth": lw}
    ax, tt_det_ell = vvis.plot_sky_sources(tt_det, src_id=None,src_kwargs=kdet_ell, sky_region_wcs = (sky_art,wcs_art), only_selected=True)
    
    sel_cs = tt_det["class_star"]<0.5
    tt_det["sel"]  = sel_cs
    kdet_cs = {"marker":"o","color":"g","markersize": 5,"alpha":1,"markeredgewidth": lw}
    ax, tt_det_cs = vvis.plot_sky_sources(tt_det, src_id=None,src_kwargs=kdet_cs, sky_region_wcs = (sky_art,wcs_art), only_selected=True)
    
    sel_art = tt_det["artifacts"]>0
    tt_det["sel"]  = sel_art
    kdet_art = {"marker":"o","color":"w","markersize": 3.5,"alpha":1,"markeredgewidth": lw}
    ax, tt_det_art = vvis.plot_sky_sources(tt_det, src_id=None,src_kwargs=kdet_art, sky_region_wcs = (sky_art,wcs_art), only_selected=True)
    plt.show()
    
    #Draw some distributions in field
    fig_dist, axs = plt.subplots(1,4,figsize=(12, 3.5),squeeze=False)
    axs = axs.flatten()
    plot_vars = ["ellip_world","size_world","class_star","artifacts"]
    plot_bins = [20           ,20          , 20         , "sqrt"]
    plot_log  = [False        , False       , False      , True ]
    for ii in range(0,4):
        axs[ii].hist([fd.tt_detections[plot_vars[ii]],tt_det[plot_vars[ii]]],density=True, bins = plot_bins[ii], log = plot_log[ii])
        #axs[ii].hist()
        axs[ii].set_xlabel(plot_vars[ii])
    
    plt.show()
    display(tt_det)
widgets.interactive( plot_art, art_idx=widgets.IntSlider(min=0, max=len(dd_arts)-1, value=0, step=1))

interactive(children=(IntSlider(value=0, description='art_idx', max=7), Output()), _dom_classes=('widget-inter…

Interresting cases (ICCS):
* 241.49, 55.50: Extended sourse cut out by class_star cut, but not ellipse

In [4]:
#selections = { "table":"tt_detections","presel_type":"and", "sel_type":"and",
#              "range": {"s2n":[3.5,np.inf],"r_fov":[0.0,0.55], "class_star": [0.6,1.0] ,"chkobj_type": [-0.5,0.5]}}
#print(selections)
#fd.select_rows(selections, remove_unselected=False)